# SPARQL generation via Generative AI
---
This notebook demonstrates one way to generate SPARQL queries from natural language questions. Here we focus on
prompting the model by implicitly showing it the database schema.

If you are running this notebook outside of an AWS environment (e.g., on your laptop) then you should uncomment
this cell and add the appropriate authentication keys:

In [ ]:
# %env AWS_ACCESS_KEY_ID=<...>
# %env AWS_SECRET_ACCESS_KEY=<...>
# %env AWS_SESSION_TOKEN=<...>

If you are running this notebook inside of an AWS environment (e.g., inside Sagemaker Studio) then
use the "conda_pytorch_p310" kernel and uncomment the following cell:

In [ ]:
# %pip install -q boto3==1.34.*
# %pip install -q botocore==1.34.*
# %pip install -q jupyter==1.0.*
# %pip install -q sagemaker==2.212.*
# %pip install -q jinja2==3.1.*
# %pip install -q ipykernel==6.29.*

In [ ]:
from pathlib import Path
import json

import boto3
import botocore
from botocore.exceptions import ClientError
import sagemaker
import jinja2

## Set Neptune and Bedrock clients

### Get connection to Neptune database
And define a function to run a SPARQL query on that database

In [ ]:
import json
import os
import requests
import boto3
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

def get_neptune_env(var):
    return os.popen(f"source ~/.bashrc ; echo ${var}").read().split("\n")[0]
    
# Grab Neptune cluster host/port from notebook instance environment variables
# Or manually set these values
GRAPH_NOTEBOOK_HOST= get_neptune_env("GRAPH_NOTEBOOK_HOST")
GRAPH_NOTEBOOK_PORT= get_neptune_env("GRAPH_NOTEBOOK_PORT")
GRAPH_NOTEBOOK_AUTH_MODE= get_neptune_env("GRAPH_NOTEBOOK_AUTH_MODE")
AWS_REGION= get_neptune_env("AWS_REGION")
USE_IAM_AUTH = GRAPH_NOTEBOOK_AUTH_MODE != 'DEFAULT'

NEPTUNE_ENDPOINT = f"https://{GRAPH_NOTEBOOK_HOST}:{GRAPH_NOTEBOOK_PORT}/sparql" 
session = boto3.Session()
 
def execute_sparql(query, crud_type='query'):
    request_data = {crud_type: query}
    data = request_data
    request_hdr = None

    if USE_IAM_AUTH:
        credentials = session.get_credentials()
        credentials = credentials.get_frozen_credentials()
        access_key = credentials.access_key
        secret_key = credentials.secret_key
        service = "neptune-db"
        session_token = credentials.token
        params = None
        creds = SimpleNamespace(
            access_key=access_key,
            secret_key=secret_key,
            token=session_token,
            region=AWS_REGION
        )
        request = AWSRequest(
            method="POST", url=NEPTUNE_ENDPOINT, data=data, params=params
        )
        SigV4Auth(creds, service,AWS_REGION).add_auth(request)
        request.headers["Content-Type"] = "application/x-www-form-urlencoded"
        request_hdr = request.headers
    else:
        request_hdr = {}
        request_hdr["Content-Type"] = "application/x-www-form-urlencoded"

    queryres = requests.request(
        method="POST", url=NEPTUNE_ENDPOINT, headers=request_hdr, data=data
    )
    if str(queryres.status_code) != "200":
        print(f"Query error {queryres.status_code} {queryres.text}")
        print(f"Here is the query:\n{query}\n")
        print(f"CRUD type is *{crud_type}*")
        print(f"Here is the result:\n{queryres.text}\n")
        raise Exception({queryres.status_code, queryres.text})

    try:
        json_resp = json.loads(queryres.text)    
        if 'results' in json_resp:
            return json_resp['results']['bindings']
        else:
            return json_resp
    except Exception as e:
        print("Exception: {}".format(type(e).__name__))
        print("Exception message: {}".format(e))
        print(f"Here is the query:\n{query}\n")
        print(f"CRUD type is *{crud_type}*")
        print(f"Here is the result:\n{queryres.text}\n")
        raise e



### Setup Bedrock client
And specify which model to use

In [ ]:
sess = sagemaker.Session()
region = sess.boto_region_name
sm_client = boto3.client("sagemaker", region_name=region)
bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)
bedrock = boto3.client("bedrock", region_name=region)

jenv = jinja2.Environment(trim_blocks=True, lstrip_blocks=True)
model_id = "anthropic.claude-v2:1"
#model_id = "anthropic.claude-3-haiku-20240307-v1:0"
#model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
temperature= 0.1

In [ ]:
def run_bedrock(prompt: str) -> str:
    try:
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(
                {
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 1024,
                    "temperature": temperature,
                    "messages": [
                        {
                            "role": "user",
                            "content": [{"type": "text", "text": prompt}],
                        }
                    ],
                }
            ),
        )
        result = json.loads(response.get("body").read())
        output_list = result.get("content", [])
        return "".join(output["text"] for output in output_list if output["type"] == "text")

    except ClientError as err:
        print(f"Error invoking {model_id}: {err.response['Error']['Code']} {err.response['Error']['Message']}")
        raise err

### Define functions to generate SPARQL using LLM via Bedrock and run it against Neptune

In [ ]:
prompt_template = (Path.cwd() / "resources" / "prompt.txt").read_text()

def generate_sparql_query(question: str) -> str:
    prompt = jenv.from_string(prompt_template).render(question=question)
    response = run_bedrock(prompt).strip()
    idx = response.index("<sparql>")
    if idx is not None:
        response = response[idx+8:]
    idx = response.index("</sparql>")
    if idx is not None:
        response = response[:idx]
    return response
   
prefixes = None
with open("resources/prefixes.txt") as f:
    prefixes = f.read()

def run_generated_query(nonpfx_query):
    query = f'''
{prefixes}

{nonpfx_query}
    '''
    return execute_sparql(query)
    
def generate_and_run(question: str):
    nonpfx_q = generate_sparql_query(str)
    print(nonpfx_q)
    return run_generated_query(nonpfx_q)
    

## Try queries
Examples:
```
---
- What GO terms are associated with human proteins?
- What GO terms are associated with human proteins? Show me their names also.
- How many citations are there for papers by A. Bairoch?
- Show me all citations by A. Bairoch
- Show me all proteins that are located in the mitochondrian
- I'd like to see the entries for all proteins encoded by the gene FNDC3A
- Select all taxa from the UniProt taxonomy
- Select all taxa from the UniProt taxonomy; show me at most 7
- Show me at most 5 taxa from the UniProt taxonomy
- Select all bacterial taxa and their scientific names from the UniProt taxonomy
- Show me up to 10 human taxa and their scientific names from the UniProt taxonomy
- Select up to 10 bacterial taxa and their scientific names from the UniProt taxonomy
- Tell me all the different categories of databases
- Tell me all the different databases you know about
- Select all UniProt entries, and their organism and amino acid sequences (including isoforms), for _E. coli K12_ and all its strains
- Select the UniProt entry with the mnemonic 'A4_HUMAN'
- Select a mapping of UniProt to PDB entries using the UniProt cross-references to the PDB database
- Select all cross-references to external databases of the category '3D structure databases' of UniProt entries that are classified with the keyword 'Acetoin biosynthesis (KW-0005)'
- Select reviewed UniProt entries (Swiss-Prot), and their recommended protein name, that have a preferred gene name that contains the text 'DNA'
- Select reviewed UniProt entries (Swiss-Prot), and their recommended protein name, that have a preferred gene name that contains the word DNA. Show me the gene name too
- Show me the preferred gene name and disease annotation of all human UniProt entries that are known to be involved in a disease
- Select all human UniProt entries with a sequence variant that leads to a 'loss of function'
- Select all distinct human UniProt entries with a sequence variant that leads to a 'loss of function', show me the text of the annotation also
- Show me all human UniProt entries with a sequence variant that leads to a tyrosine to phenylalanine substitution
- Show me all human UniProt entries with a sequence variant that leads to a Tyr to phenylalanine substitution
- Select all UniProt entries with annotated transmembrane regions and the regions' begin and end coordinates on the canonical sequence
- Select all UniProt entries that were integrated on the 30th of November 2010
- Select all UniProt entries that were integrated on or before the 30th of November 2010
- Select all UniProt entries that were integrated on the month of November 2010
- Show me all UniProt entries that were added to the database on the month of November 2010
#    "Was any UniProt entry integrated on the 9th of January 2013?
#    "Select all triples that relate to the EMBL CDS entry AA089367.1
#    "Select all triples that relate to the taxon that describes Homo sapiens in the named graph for taxonomy
- Select the average number of cross-references to the PDB database of UniProt entries that have at least one cross-reference to the PDB database
- Select the number of UniProt entries for each of the EC (Enzyme Commission) second level categories
- Find all Natural Variant Annotations if associated via an evidence tag to an article with a pubmed identifier.
#- Find how often an article in pubmed was used in an evidence tag in a human protein (ordered by most used to least) # timesout
- Find where disease related proteins are known to be located in the cell
#- For two accessions find the GO term labels and group them into GO process,function and component
- How many reviewed entries (Swiss-Prot) are related to kinase activity?
#- Find the release number of the uniprot data that is currently being queried
#- Find any uniprot entry which has a name 'HLA class I histocompatibility antigen, B-73 alpha chain' # https://docs.streamlit.io/library/advanced-features/session-state
#- Find any uniprot entry, or an uniprot entries domain or component which has a name 'HLA class I histocompatibility antigen, B-73 alpha chain'
#- Construct new triples of the type 'HumanProtein' from all human UniProt entries
- list all the Homo Sapiens proteins classified with "cholesterol biosynthetic process"
- list all the Homo Sapiens proteins classified with "cholesterol biosynthetic process". Include their names.
- list all the Homo Sapiens proteins classified with "cholesterol biosynthetic process". Don't include their names.
- find all the Homo Sapiens related proteins that have a Gene Ontology (GO) code
- look within the taxonomy tree, to see if there are any subclass records under Homo Sapiens. Return the scientific name also
```

In [ ]:
generate_and_run("Show me all proteins that are located in the mitochondrian")


In [ ]:
q='''Select the average number of cross-references to the PDB database of UniProt entries that have at least one cross-reference to the PDB database'''
generate_and_run(q)